In [ ]:
!export TOKENIZERS_PARALLELISM=false
!python -m pip install --upgrade pip
!pip install typing_extensions pydantic openai
!pip install datasets transformers peft trl bitsandbytes
!pip install sentence-transformers langchain langchain_community
!pip install chromadb
#!pip uninstall numpy -y
# !pip install numpy==1.26.4
!pip install "numpy<2"

In [ ]:
from huggingface_hub import login
login(token='')

from peft import PeftModel

from transformers import AutoModelForCausalLM

model_name = "google/gemma-3-4b-it"

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "jeongyoonhuh/q_lora_korqa")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: Loading a multimodal model with `AutoModelForCausalLM` is deprecated and will be removed in v5. `AutoModelForCausalLM` will be used to load only the text-to-text generation module.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager
from langsmith import Client
import os
import uuid
import time
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pipeline 래핑
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.1,
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 임베딩 및 벡터스토어 로드
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
vector_store = Chroma(
    persist_directory="chroma_index",
    embedding_function=embedding
)
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.7
    }
)

# ✅ 사용자 정의 프롬프트
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        너는 항공기 반입 금지 및 제한 물품에 대하여 친절하게 답해주고 나라별 여행지 추천 및 문화차이를 잘 설명할 수 있어
        문맥에 해당 하는 내용들만 신뢰도있게 대답해야해

        ### 질문:
        {question}
        
        ### 문맥:
        {context}

        ### 답변:
    """
)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

In [9]:
os.environ["LANGCHAIN_API_KEY"] = "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"  # 실제 API 키로 변경
os.environ["LANGCHAIN_PROJECT"] = "travel_sj_project"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

client = Client()

# 메모리 객체 생성
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 대화형 QA 체인 생성
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt}
)

/tmp/ipykernel_7838/1065027196.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
# ✅ 국가 목록 상수 추가
SUPPORTED_COUNTRIES = [
    "한국", "일본", "베트남", "미국", "필리핀", "싱가포르", "대만",
    "인도네시아", "홍콩", "중국", "말레이시아"
]

def detect_country(query):
    """사용자 질문에서 국가 감지"""
    query_lower = query.lower()

    # ✅ 기존 조건 → 딕셔너리로 정리
    country_map = {
        "일본": ["일본", "재팬", "도쿄", "교토", "오사카"],
        "대만": ["대만", "타이완", "타이페이"],
        "한국": ["한국", "서울", "부산", "제주"],
        "베트남": ["베트남", "하노이", "호치민", "다낭", "하롱베이"],
        "미국": ["미국", "뉴욕", "로스앤젤레스", "시카고", "라스베가스", "하와이"],
        "필리핀": ["필리핀", "마닐라", "세부", "보라카이", "팔라완"],
        "싱가포르": ["싱가포르", "센토사", "마리나베이", "가든스바이더베이"],
        "인도네시아": ["인도네시아", "발리", "자카르타", "롬복"],
        "홍콩": ["홍콩", "침사추이", "란타우", "빅토리아피크"],
        "중국": ["중국", "베이징", "상하이", "광저우", "시안", "만리장성"],
        "말레이시아": ["말레이시아", "쿠알라룸푸르", "페낭", "코타키나발루", "랑카위"]
    }

    for country, keywords in country_map.items():
        if any(keyword in query_lower for keyword in keywords):
            return country

    return "Unknown"  # ✅ 감지되지 않으면 Unknown 반환


def answer_question(query, qa_chain, vector_store, chat_history=None):
    """국가 필터링을 적용한 질문 처리"""

    # 국가 감지
    country = detect_country(query)

    # ✅ 없는 나라 처리 추가
    if country == "Unknown":
        return {
            "result": "관련 데이터가 없습니다.",
            "source_documents": [],
            "question": query
        }

    try:
        is_conv_chain = hasattr(qa_chain, 'memory')

        country_keywords = {
            '일본': ['일본', '재팬', '도쿄'],
            '대만': ['대만', '타이완', '타이페이'],
            '한국': ['한국', '서울', '부산'],
            '베트남': ['베트남', '하노이', '호치민'],
            '미국': ['미국', '뉴욕', '워싱턴'],
            '필리핀': ['필리핀', '마닐라', '세부'],
            '싱가포르': ['싱가포르', '센토사'],
            '인도네시아': ['인도네시아', '발리', '자카르타'],
            '홍콩': ['홍콩'],
            '중국': ['중국', '베이징', '상하이'],
            '말레이시아': ['말레이시아', '쿠알라룸푸르']
        }

        enhanced_query = f"{country_keywords.get(country, [country])[0]} {query}"

        if is_conv_chain:
            if chat_history is not None:
                result = qa_chain({"question": enhanced_query, "chat_history": chat_history})
            else:
                result = qa_chain({"question": enhanced_query})
        else:
            retriever = vector_store.as_retriever(search_kwargs={"k": 5})
            docs = retriever.get_relevant_documents(enhanced_query)

            keywords = country_keywords.get(country, [country.lower()])
            filtered_docs = [
                doc for doc in docs
                if any(keyword.lower() in doc.metadata.get('source', '').lower() or
                       keyword.lower() in doc.page_content.lower()
                       for keyword in keywords)
            ]

            if not filtered_docs:
                filtered_docs = docs

            result = qa_chain({
                "input_documents": filtered_docs,
                "query": query
            })

        return {
            "result": result.get('result',
                      result.get('output_text',
                      result.get('answer', str(result)))),
            "source_documents": result.get('source_documents', []),
            "question": query
        }

    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        return {
            "result": f"처리 중 오류가 발생했습니다: {str(e)}",
            "source_documents": [],
            "question": query
        }

In [12]:
# 첫 번째 질문
query = "대만에 소세지 들고가도돼??"
result = answer_question(query, qa_chain, vector_store)
print("Answer:", result.get('result', '응답 없음'))

# 후속 질문
query = "그럼 육포는 어때?"
result = answer_question(query, qa_chain, vector_store)
print("Answer:", result.get('result', '응답 없음'))

/tmp/ipykernel_7838/4169818662.py:70: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": enhanced_query})


Answer: 
        너는 항공기 반입 금지 및 제한 물품에 대하여 친절하게 답해주고 나라별 여행지 추천 및 문화차이를 잘 설명할 수 있어
        문맥에 해당 하는 내용들만 신뢰도있게 대답해야해

        ### 질문:
        대만 대만에 소세지 들고가도돼??
        
        ### 문맥:
        한국하고 상당히 가깝고 유교사상 때문에 문화와 생활(특히 예절)이 한국과 거의 다를 것이 없다고 생각할 수도 있지만, 대만에 살면서 느끼는 건 한국과 대만은 너무나도 다르다는 겁니다.  그래서 현지인들의 문화를 잘 모르는 여행객들은 사소한 실수 때문에 마찰이 일어날 수도 있답니다.  즐겁게 놀러온 여행지에서 이런 일이 있으면 절대 안 되겠죠? 가깝지만 더욱 먼 나라 대만 그래서 오늘은 타이완몬스터가 한국과 다른 대만의 문화! 그리고 대만에서 해서는 안 되는 행동들에 대해서 간단히 알려드리겠습니다! 한국에서는 인사를 나눌 때 고개를 숙이지만 대만에서는 보통 악수를 나누며 안부를 묻습니다.  비교적 젊은 사람이 나이 드신 분에게 악수를 청하는 모습은 정말 한국에서는 보기 힘든 모습이죠 식사 시에 밥그릇을 들고 먹는다 한국에서 밥그릇을 들고 먹는 행동을 부모님들에게 보인다면 보통 잔소리로 끝날 문제가 아니죠ㅎㅎ 저도 어릴 적 만화영화를 보고 밥그릇을 들고 먹다가 부모님께 혼난 적이 있는데요. 우리나라의 식사예절과는 반대로 대만에서는 밥그릇을 꼭 손에 들고 먹는답니다. 왜냐하면 대만은 식탁에서 음식을 흘리면서 먹는 것을 커다란 실례라고 생각하기 때문이죠. 그렇기 때문에 밥그릇을 입에 꼭 데고 먹는 대만 사람들이 많이 있답니다. 하지만 그렇다고 탕 을 입에 가져가서 드시면 안 돼요! 탕은 반드시 숟가락으로 드셔야 한답니다! 일본은 밥그릇과 국물요리가 들어있는 그릇을 들고 먹지만 대만은 밥그릇과 반찬이 있는 접시만 들고 먹는답니다. 신기하죠? 대중교통에서 취식 불가능 대만에 놀러 오는 여행객들이 가장 많이 하는 실수 BEST1!

In [13]:
query = "아프리카에 반입금지 물품이 뭐야??"
result = answer_question(query, qa_chain, vector_store)
print("Answer:", result.get('result', '응답 없음'))  # 🔥 관련 데이터가 없습니다. 출력됨

query = "아프리카에 지켜야 할 에티켓이 뭐야??"
result = answer_question(query, qa_chain, vector_store)
print("Answer:", result.get('result', '응답 없음'))  # 🔥 관련 데이터가 없습니다. 출력됨

Answer: 관련 데이터가 없습니다.
Answer: 관련 데이터가 없습니다.


In [14]:
!pip install gradio

In [15]:
import gradio as gr

def gradio_answer_question(query):
    result = answer_question(query, qa_chain, vector_store)
    return result.get('result', '응답 없음')

iface = gr.Interface(
    fn=gradio_answer_question,
    inputs=gr.Textbox(lines=2, placeholder="여행 관련 질문을 입력하세요."),
    outputs=gr.Textbox(),
    title="여행 정보 챗봇",
    description="국가별 반입 금지 품목 및 문화 차이에 대해 질문하세요."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c7ce851a21e85afa63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
